In [1]:
import json
import numpy as np
from tqdm import tqdm
from pprint import pprint
from sklearn.cluster import KMeans

from label_processor import CStdLib

In [2]:
with open('result_logits.json', encoding='utf-8') as f:
    logits_json = json.load(f)

In [3]:
embed = CStdLib(single=False)

In [4]:
pbar = tqdm(logits_json.items())
i = 0
for key, prod_dict in pbar:
    optional_tags = prod_dict['optional_tags']
    num_class = len(optional_tags)
    imgs_tags = prod_dict['imgs_tags']
    imgs_logits = []
    for i in range(len(imgs_tags)):
        imgs_logits.append(np.fromstring(imgs_tags[i][key + '_{}.jpg'.format(i)], dtype=np.float32, sep=' '))
    imgs_logits = np.array(imgs_logits)
    cluster = KMeans(n_clusters=num_class, random_state=42).fit(imgs_logits)
    labels = cluster.labels_
    match_score = np.zeros((num_class, num_class))
    # match_score[i, j] 表示 kmeans 分类结果为 i 和 optional_tags[j] 的匹配度
    embeded_tags = [embed(tag) for tag in optional_tags]
    not_matched = False
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        logits = imgs_logits[id_i]
        for j in range(num_class):
            logits_ids = embeded_tags[j]
            if len(logits_ids) > 0:
                logits_j = logits[:, logits_ids]
                match_score[i, j] = np.mean(logits_j)
            else:
                not_matched = True
                match_score[i, j] = 1
    # 对 scores 进行归一化，使得每一列的 ord 范数为 1；然后在每一行中找到最大值，即为该行商品对应的标签。
    # 按列归一化，按行找最大值，这样就做到了双向选择。
    # 使用较高阶的范数能增大未匹配标签对应的概率，有利于提高 acc/em。
    match_score_norm = match_score / np.linalg.norm(match_score, axis=0, ord=5)
    # print(key, '\n', match_score_norm)
    label_id = np.argmax(match_score_norm, axis=1)
    # print(label_id)
    for i in range(num_class):
        id_i = np.where(labels == i)[0]
        for j in id_i:
            logits_json[key]['imgs_tags'][j][key + '_{}.jpg'.format(j)] = optional_tags[label_id[i]]
    if not_matched:
        print(key, '\n', match_score, '\n', match_score_norm)
        pprint(logits_json[key])

  5%|▌         | 272/5331 [00:03<00:56, 89.96it/s] 

624952669722 
 [[3.6693418  1.        ]
 [6.44651413 1.        ]] 
 [[0.56262988 0.87055056]
 [0.98846105 0.87055056]]
{'imgs_tags': [{'624952669722_0.jpg': '珊瑚色组合'},
               {'624952669722_1.jpg': '珊瑚色组合'},
               {'624952669722_2.jpg': '橄榄色组合'},
               {'624952669722_3.jpg': '橄榄色组合'},
               {'624952669722_4.jpg': '橄榄色组合'}],
 'optional_tags': ['珊瑚色组合', '橄榄色组合']}


  7%|▋         | 353/5331 [00:03<00:45, 109.17it/s]

614043736149 
 [[ 7.20567131  3.90869713  1.96744847  1.          4.77405357]
 [ 3.43354893  2.63160849 10.63453388  1.          3.20032692]
 [ 3.47113943 10.02686214  2.45175362  1.          2.63745213]
 [ 3.31556606  3.57004356  2.21922946  1.         12.93972588]
 [12.69450951  3.98268509  3.5317955   1.          4.43831062]] 
 [[0.56071533 0.38783741 0.18480999 0.72477966 0.36800376]
 [0.26718448 0.26111929 0.9989426  0.72477966 0.24669441]
 [0.27010961 0.99490755 0.23030263 0.72477966 0.2033057 ]
 [0.25800354 0.35423478 0.20846074 0.72477966 0.99744749]
 [0.98783386 0.39517881 0.33175511 0.72477966 0.34212331]]
{'imgs_tags': [{'614043736149_0.jpg': '条纹'},
               {'614043736149_1.jpg': '桔色'},
               {'614043736149_2.jpg': '红色'},
               {'614043736149_3.jpg': '桔色'},
               {'614043736149_4.jpg': '红色'},
               {'614043736149_5.jpg': '黑色'},
               {'614043736149_6.jpg': '条纹'},
               {'614043736149_7.jpg': '黑色'},
               {

 15%|█▌        | 811/5331 [00:08<00:46, 96.48it/s] 

...

 85%|████████▍ | 4524/5331 [00:47<00:07, 105.01it/s]

577541680180 
 [[15.08464813  1.          7.01837683  2.43292999]
 [ 2.56085491  1.          2.59838891 12.8776989 ]
 [ 3.69489646  1.         11.64810181  3.12644339]
 [ 4.60691595  1.          4.84250879  9.02220917]] 
 [[0.99926569 0.75785828 0.59197889 0.18308921]
 [0.16964098 0.75785828 0.21916626 0.96910628]
 [0.2447643  0.75785828 0.98248221 0.2352793 ]
 [0.30518001 0.75785828 0.40845099 0.67896288]]
{'imgs_tags': [{'577541680180_0.jpg': '红色'},
               {'577541680180_1.jpg': '红色'},
               {'577541680180_2.jpg': '蓝色'},
               {'577541680180_3.jpg': '蓝色'},
               {'577541680180_4.jpg': '橙色'},
               {'577541680180_5.jpg': '深色'},
               {'577541680180_6.jpg': '橙色'},
               {'577541680180_7.jpg': '蓝色'},
               {'577541680180_8.jpg': '蓝色'}],
 'optional_tags': ['橙色', '深色', '红色', '蓝色']}


 89%|████████▉ | 4765/5331 [00:49<00:06, 89.69it/s] 

612126042525 
 [[ 1.         12.58297062]
 [ 1.         13.06167889]] 
 [[0.87055056 0.85370578]
 [0.87055056 0.88618428]]
{'imgs_tags': [{'612126042525_0.jpg': '6802格子'},
               {'612126042525_1.jpg': '6802格子'},
               {'612126042525_2.jpg': '957黑色'},
               {'612126042525_3.jpg': '6802格子'},
               {'612126042525_4.jpg': '6802格子'},
               {'612126042525_5.jpg': '6802格子'},
               {'612126042525_6.jpg': '6802格子'}],
 'optional_tags': ['6802格子', '957黑色']}


100%|██████████| 5331/5331 [00:56<00:00, 94.80it/s] 


In [5]:
with open('./result_labels.json', 'w') as f:
    json.dump(logits_json, f, indent=4, ensure_ascii=False)